# LSQT-Jupyter

## 1. Introduction

* `LSQT-Jupyter` is a Jupyter notebook demonstrating some practical apects of the linear-scaling quantum transport methods reviewd in Ref. [1].
* The programming language chosen is `Python 3`.
* We will refer to Ref. [1] frequently for the relevant theoretical backgounds.
* This code is only suitable for leanring the methods. For high performance computing, check the `GPUQT` code I wrote: https://github.com/brucefan1983/gpuqt
* This code only considers square lattice with Anderson disoder as the model system. For more general simulations, again check the `GPUQT` code.
* For fans of `Matlab`, check the `lsqt-matlab` code I wrote: https://github.com/brucefan1983/lsqt-matlab

[1] Zheyong Fan, Jose Hugo Garcia, Aron W Cummings, Jose-Eduardo Barrios, Michel Panhans, Ari Harju, Frank Ortmann, and Stephan Roche, Linear Scaling Quantum Transport Methodologies, submitted to Reviews of Modern Physics. https://arxiv.org/abs/1811.07387

## 2. Unit system (We are working in 2D)
* Basic units:
 * Reduced Planck constant $\hbar$ = 1
 * Elementary charge $e$ = 1
 * Energy unit $\gamma=1$ is hopping integral
 * Length unit $a=1$ is the lattice constant
* Derived units:
 * Time: $\hbar/\gamma$
 * Densit of states (DOS) in 2D: $1/\gamma/a^2$
 * Velocity autocorrelation (VAC): $a^2\gamma^2/\hbar^2$
 * Mean square displacement (MSD): $a^2$
 * Electrical conductivity in 2D: $e^2/\hbar$
 * Electrical conductance: $e^2/\hbar$

## 3. Import the needed Python packages

In [1]:
import numpy as np              # used frequently
import matplotlib as mpl        # do we need this?
import matplotlib.pyplot as plt # used for plotting
from scipy import sparse        # crucial for performance
from scipy import integrate     # do we need this?
from scipy import special       # for Bessel function

## 4. Construct the tight-binding model
* We consdier the nearest-neighbor tight-bindng model defined on a square lattice with lattice constant $a$ and dimensions $N=N_x\times N_y$.
* The Hamiltonian can be written as
\begin{equation}
\hat{H} = \sum_{ij} (-\gamma) c_i^{\dagger} c_j + \sum_{i} U_i c_i^{\dagger} c_i,
\end{equation}
where $-\gamma$ is the hopping integral and $U_i$ are the on-site potentials. The on-site potentials are uniformly distributed in an interval $[-W/2, W/2]$, where $W$ is the Anderson disorder strength. For simplicity, we consider open boundary conditions in the $y$ direction and study the transorpt in the $x$ direction.

### 4.1 Define an indexing scheme
* The purpose is to map th 2D indices in the lattice into 1D indices
* The 1D indices will be used for the Hamiltionian, velcoity operator, and various vectors.
* This only requires a simple function shown below:

In [2]:
def find_index(nx, ny, Ny):
    """
    Get a 1D index from a 2D index
    Args:
        nx (int): x-index of a site
        ny (int): y-index of the same site
    Returns:
        index (int): 1D index for (nx, ny)
    """
    index = nx * Ny + ny
    return index

### 4.2. Define the function for creating the sparse Hamiltonian and velocity operator
* We consider square lattice with $N_x \times N_y$ lattice sites.
* Use periodic boundary conditions in the $x$ direction.
* Use open boundary conditions in the $y$ direction.
* Assume the transport is in the $x$ direction.
* All the hoppings integral $\gamma=1$ defines the energy units.
* The lattices constant $a=1$ defines the length units.
* Anderson diorder strength is $W$, which means that random on-site potentials are uniformly chosen from $[-W/2, W/2]$. $W$ is in units of $\gamma$.

In [11]:
def find_H(Nx, Ny, W):
    """
    Construct the sparse Hamiltonian and velocity operator
    Args:
        Nx (int): number of lattice points in the x direction
        Ny (int): number of lattice points in the y direction
        W (real): Anderson disorder strength
    Returns:
        H (sparse.csr_matrix): The sparse Hamiltonian
        V (sparse.csr_matrix): The sparse velocity operator
    """
    N = Nx * Ny; # total number of sites
    row_H = np.zeros(N * 4 - Nx * 2) # the row indices for H
    col_H = np.zeros(N * 4 - Nx * 2) # the column indices for H
    Hij = -np.ones(N * 4 - Nx * 2, complex) # nonzero Hamiltonian matrix elements
    row_V = np.zeros(N * 2) # row indices for V
    col_V = np.zeros(N * 2) # column indices for V
    Vij = np.zeros(N * 2, complex) # nonzero velocity matrix elements
    row_U = np.arange(0, N) # row and column indices for U
    Uij = np.random.uniform(-W * 0.5, W * 0.5, N) # on-site potentials
    count_H = 0 # number of nonzero H elements
    count_V = 0 # number of nonzero V elements
    for nx in range(Nx):
        for ny in range(Ny):
            # (0) # get the index of the center site
            index_center = find_index(nx, ny, Ny)
            # (1) consider the left neighbor (periodic boundary)
            index_left = find_index((nx - 1) % Nx, ny, Ny)
            row_H[count_H] = index_center
            col_H[count_H] = index_left
            count_H += 1
            row_V[count_V] = index_center
            col_V[count_V] = index_left
            Vij[count_V] = 1j
            count_V += 1
            # (2) consider the right neighbor (periodic boundary)
            index_right = find_index((nx + 1) % Nx, ny, Ny)
            row_H[count_H] = index_center
            col_H[count_H] = index_right
            count_H += 1
            row_V[count_V] = index_center
            col_V[count_V] = index_right
            Vij[count_V] = -1j
            count_V += 1
            # (3) consider the upper neighbor (open boundary)
            if ny < Ny - 1:
                index_up = find_index(nx, (ny + 1), Ny)
                row_H[count_H] = index_center
                col_H[count_H] = index_up
                count_H += 1
            # (4) consider the down neighbor (open boundary)
            if ny > 0:
                index_down = find_index(nx, (ny - 1), Ny)
                row_H[count_H] = index_center
                col_H[count_H] = index_down
                count_H += 1
    H = sparse.csr_matrix((Hij, (row_H, col_H)), shape = (N, N))
    U = sparse.csr_matrix((Uij, (row_U, row_U)), shape = (N, N))
    H = H + U
    V = sparse.csr_matrix((Vij, (row_V, col_V)), shape = (N, N))
    return (H, V)